In [ ]:
target = 'area'

# Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

import statsmodels.api as sm
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols
from scipy.stats import zscore
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

---

# Load and model data
## Run the below cells before starting with the project

In [1]:
# path = 'forestfires.csv'
path = " "                                                       #Input path for file location
df = pd.read_csv(path)

NameError: name 'pd' is not defined

In [ ]:
out_columns = ['area','FFMC','ISI','rain']
df = pd.get_dummies(df,columns=['day','month'],drop_first=True)
np.log1p(df[out_columns]).skew(), np.log1p(df[out_columns]).kurtosis()
mask = df.loc[:,['FFMC']].apply(zscore).abs() < 3
df['rain'] = df['rain'].apply(lambda x: int(x > 0.0))
df = df[mask.values]
out_columns.remove('rain')
df[out_columns] = np.log1p(df[out_columns])
df[out_columns].skew()

In [ ]:
# a categorical variable based on forest fire area damage
# No damage, low, moderate, high, very high
def area_cat(area):
    if area == 0.0:
        return "No damage"
    elif area <= 1:
        return "low"
    elif area <= 25:
        return "moderate"
    elif area <= 100:
        return "high"
    else:
        return "very high"

df['damage_category'] = df['area'].apply(area_cat)
df.head()

In [ ]:
out_columns = ['area','FFMC','ISI','rain']
df = pd.get_dummies(df,columns=['day','month'],drop_first=True)

In [ ]:
X = df.drop(columns=['area','damage_category'])
y = df['area']



# Checking assumptions for linear regression in statistics

1. Linearity of model
    
2. Normality of residuals

3. Homoscedasticity

4. No Autocorrelation

5. Multicollinearity 

In [ ]:
X_constant = sm.add_constant(X)

# Build OLS model
                                                              # Type code here and also get summary

## 1. Linearity of residuals


**Linearity can be measured by two methods:**

- Plot the observed values Vs predicted values` and plot the `Residual Vs predicted values` and see the linearity of residuals. 
-  Rainbow test

**Rainbow test**

In [ ]:
import scipy.stats as stats
import pylab

# get an instance of Influence with influence and outlier measures 
st_resid =                                                                                 #Type Code here




- **Null hypothesis (H0):** The Null hypothesis is that the regression is correctly modeled as linear.
- **Alternate hypothesis(H1)**: The model is non-linear

In [ ]:
# return fstat and p-value
                                                                                           # Type code here

**Expectation Mean of residual is zero**

In [ ]:
# The mean expected value around 0, it implies linearity is preserved
                                                                                   # Calculate mean, type code here

In [ ]:
def linearity_test(model, y):
    '''
    Function for visually inspecting the assumption of linearity in a linear regression model.
    It plots observed vs. predicted values and residuals vs. predicted values.
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2,figsize=(15,5))

    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Observed vs. Predicted Values', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Residuals vs. Predicted Values', fontsize=16)
    ax[1].set(xlabel='Predicted', ylabel='Residuals')
    


In [ ]:
# Test linearity using above function 
linearity_test(lin_reg, y) 
plt.tight_layout()

The desired outcome of plots is that points are symmetrically distributed around a diagonal line in the former plot or around horizontal line in the latter one. 

> - By observing  the plots the linearity assumption is not there 
- Adding new features might result in linearity of model 
- Also, transforming the feature from non-linear to linear using various data transformation techniques can help.

## 2. Normality of the residuals

In [ ]:
sns.distplot(lin_reg.resid,fit=stats.norm)
plt.text(4,0.5,f"Skewness: {round(lin_reg.resid.skew(),2)}",fontsize=15)
plt.show()

**Test for normality: Jarque Bera**

For a good model, the residuals should be normally distributed.
The higher the value of Jarque Bera test, the lesser the residuals are normally distributed.

The Jarque–Bera test is a goodness-of-fit test of whether sample data 
have the skewness and kurtosis matching a normal distribution.

The jarque bera test tests whether the sample data has the skewness and kurtosis matching a normal distribution.

Note that this test generally works good for large enough number of data samples(>2000) as the test statistics asymptotically has a chi squared distribution with degrees 2 of freedom.

**Null hypothesis (H0)** - Residuals are normally distributed

In [ ]:
# Use the Jarque Bera test 
                                                                                                # Type code here


plt.text(-2,4,f"Jarque bera: {jb}",fontsize=15)
plt.show()

> The p-value is 0 which simply means we can reject out NULL hypothesis.
We can fix that by
- Removing the outliers in the data
- Fixing the Non-linearity in our dependent or target feature
- Removing the bias, the bias might be contributing to the non-normality. 

## 3. Homoscedasticity


Homoscedacity: If the residuals are symmetrically distributed across the trend , then it is called as homoscedacious. 

Heteroscedacity: If the residuals are not symmetric across the trend, then it is called as heteroscedacious.


**Goldfeld-Quandt test for Homoscedasticity**

H0 = constant variance among residuals (Homoscedacity)

Ha = Heteroscedacity.

In [ ]:
sms.het_goldfeldquandt(lin_reg.resid, lin_reg.model.exog)

In [ ]:
model =                                                                                        #Type code here
fitted_vals =                                                                                  #Type code here
resids =                                                                                       #Type code here
resids_standardized =                                                                          #Type code here

fig, ax = plt.subplots(1,2)

# Use regplot to plot fitted values v/s residuals
                                                                                               #Type code here
    
ax[0].set_title('Predicted vs Residuals', fontsize=16)
ax[0].set(xlabel='Predicted Values', ylabel='Residuals')

# Use regplot to plot fitted values v/s root of absolute residuals
                                                                                               #Type code here
    
ax[1].set_title('Scale-Location', fontsize=16)
ax[1].set(xlabel='Predicted Values', ylabel='sqrt(abs(Residuals))')

name = ['F statistic', 'p-value']
test = sms.het_goldfeldquandt(model.resid, model.model.exog)
lzip(name, test)
plt.tight_layout()

>- To identify homoscedasticity in the plots, the placement of the points should be equally distributed, random, no pattern (increase/decrease in values of residuals) should be visible and a flat red line.
- In the plots we can see there are no paticular patterns and P-Values is also greater than 0.05 ,so we can say that there is homoscedasticity.
- Outliers can make it Heteroscedacious, Transforming (log or Box cox, if > 0) the dependent or independent variables can help fix it.

[Reference](https://datascienceplus.com/how-to-detect-heteroscedasticity-and-rectify-it/)

## 4. No Autocorrelation

Autocorrelation measures the relationship between a variable's current value and its past values.

**Test for autocorrelation : Durbin- Watson Test**

It's test statistic value ranges from 0-4. If the value is between 
- 0-2, it's known as Positive Autocorrelation.
- 2-4, it is known as Negative autocorrelation.
- exactly 2, it means No Autocorrelation.

For a good linear model, it should have low or no autocorrelation.

```python
from statsmodels.stats.stattools import durbin_watson
durbin_watson(lin_reg.resid)
```

> In our case, Durbin-Watson: 0.979

In [ ]:
import statsmodels.tsa.api as smt
# Confidence intervals are drawn as a cone. 
# By default, this is set to a 95% confidence interval, 
# suggesting that correlation values outside of this code are very likely a correlation 
# and not a statistical fluke

#Plot autocorrelation 
acf =                                                                                          #Type code here

> - By observing the above data we can say that there is positive autocorrelation is present , we can reduce it by using fine tuning our parameters
- We can even use Generalize Least Squares (GLS) model

## 5. Multicollinearity 

Multicollineariy arises when one independent variable can be linearly predicted by others with a substantial level of accuracy.

In [ ]:
plt.figure(figsize =(16,10))

# Plot heatmap using corelation matrix
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Calculate variance inflation factor
vif =                                                                                         #Type code here
pd.DataFrame({'vif': vif[1:]}, index=X.columns).sort_values(by="vif",ascending=False)

> There is multicollinearity present between some features where vif >5.
- We can even use PCA to reduce features to a smaller set of uncorrelated components.
- To deal with multicollinearity we should iteratively remove features with high values of VIF.
